In [1]:
%load_ext autoreload
%autoreload 2

from api_tricount import *


In [83]:
%load_ext autoreload
%autoreload 2
from api_portfolio import build_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
dicts = pd.DataFrame(ss.worksheet('Dict').get_all_records())
operation = pd.DataFrame(ss.worksheet('Operations').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
greenbull = pd.DataFrame(ss.worksheet('GREENBULL').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
assets = dicts.set_index('Asset')
print(assets)

market = yf.download(' '.join([*assets['Forex'], *assets['Market'][:-1]]), start='2021-04-28')['Close']
market = pd.concat([market, greenbull], axis=1).ffill().bfill()
print(market.columns)


              Market Currency     Forex           Class  PriceFmt PositionFmt  \
Asset                                                                           
EUR         EUREUR=X           EUREUR=X         Deposit                         
€           EUREUR=X      EUR  EUREUR=X            Cash  {:.0f} €      {:.0f}   
$              EUR=X      EUR  EUREUR=X            Cash  {:.4f} €      {:.0f}   
USDT           EUR=X     EUR   EUREUR=X            Cash  {:.4f} €      {:.0f}   
ESE           ESE.PA        €  EUREUR=X        Equities  {:.2f} €      {:.0f}   
PUST                        €  EUREUR=X        Equities  {:.2f} €      {:.0f}   
RS2K                        €  EUREUR=X        Equities  {:.2f} €      {:.0f}   
OBLI         OBLI.PA        €  EUREUR=X    Fixed Income  {:.2f} €      {:.0f}   
GOLD         GOLD.PA        €  EUREUR=X     Commodities  {:.2f} €      {:.0f}   
BTC          BTC-USD     USDT  EURUSD=X  Cryptocurrency  {:.0f} $      {:.4f}   
ETH                      USD

In [35]:

df = pd.DataFrame(index=market.index, columns=pd.MultiIndex(levels=[[],[],[],[]], codes=[[],[],[],[]]))#, names=['portfolio', 'class', 'asset', 'metric']))
df.head()

for portfolio in np.unique(operation['Portfolio']):
    print(portfolio)
    dfp: pd.DataFrame = operation[operation['Portfolio'] == portfolio]

    print(f"\tAssets")
    for asset in np.unique(dfp['Asset']):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{classs}:\t{asset}")

        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)

        df[portfolio, classs, asset, 'Quantity'] = dfa['Quantity']
        df[portfolio, classs, asset, 'Operation'] = dfa['Operation']
        df[portfolio, classs, asset, 'Market'] = market[assets.loc[asset]['Market']]
        df[portfolio, classs, asset, 'Forex'] = market[assets.loc[asset]['Forex']]
    
    print(f"\tCurrency")
    for currency in np.unique(dfp['Currency']):
        classs = assets.loc[currency]['Class']
        print(f"\t\t{classs}:\t{currency}")

        dfc: pd.DataFrame = dfp[dfp['Currency'] == currency]
        dfc = dfc.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)
        
        if (portfolio, classs, currency, 'Quantity') not in df.columns:
            df[portfolio, classs, currency, 'Quantity'] = 0
        if (portfolio, classs, currency, 'Operation') not in df.columns:
            df[portfolio, classs, currency, 'Operation'] = 0
        if (portfolio, classs, currency, 'Market') not in df.columns:
            df[portfolio, classs, currency, 'Market'] = market[assets.loc[currency]['Market']]

        df[portfolio, classs, currency, 'Quantity'] -= dfc['Operation']
        df[portfolio, classs, currency, 'Operation'] -= dfc['Operation'] * df[portfolio, classs, currency, 'Market']


    print(f"\tBuild datatable ...")
    for asset in np.unique([*dfp['Asset'], *dfp['Currency']]):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{asset}")

        toEUR = market[assets.loc[asset]['Forex']]
        df[portfolio, classs, asset, 'OperationEUR'] = df[portfolio, classs, asset, 'Operation'] / toEUR

        position, invested, investedEUR = pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in df.index:
            position[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Quantity'])
            invested[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Operation'])
            investedEUR[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'OperationEUR'])

        df = pd.concat([df, pd.DataFrame({
                (portfolio, classs, asset, 'Position'): position,
                (portfolio, classs, asset, 'Invested'): invested,
                (portfolio, classs, asset, 'InvestedEUR'): investedEUR,
            })], axis=1)

        df[portfolio, classs, asset, 'PRU'] = df[portfolio, classs, asset, 'Invested'] / df[portfolio, classs, asset, 'Position']
        df[portfolio, classs, asset, 'Value'] = df[portfolio, classs, asset, 'Position'] * df[portfolio, classs, asset, 'Market']
        df[portfolio, classs, asset, 'PnL'] = df[portfolio, classs, asset, 'Value'] - df[portfolio, classs, asset, 'Invested']
            
        df[portfolio, classs, asset, 'ValueEUR'] = df[portfolio, classs, asset, 'Value'] / toEUR
        df[portfolio, classs, asset, 'PnLEUR'] = df[portfolio, classs, asset, 'ValueEUR'] - df[portfolio, classs, asset, 'InvestedEUR']



DMA
	Assets
		Cash:	€
	Currency
		Deposit:	EUR
	Build datatable ...
		EUR
		€
ZEN
	Assets
		Cash:	$
		Cryptocurrency:	BTC
		Equities:	ESE
		Commodities:	GOLD
		Equities:	GREENBULL
		Fixed Income:	OBLI
		Cash:	USDT
		Cash:	€
	Currency
		Cash:	$
		Deposit:	EUR
		Cash:	USDT
		Cash:	€
	Build datatable ...
		$
		BTC
		ESE
		EUR
		GOLD
		GREENBULL
		OBLI
		USDT
		€


In [38]:

print(f"\tConcate class data ...")
for classs in np.unique(df.columns.get_level_values(1)):
    for portfolio in np.unique(operation['Portfolio']):
        if classs not in df[portfolio].columns.get_level_values(0):
            df[portfolio, classs, 'All', 'ValueEUR'] = 0
            df[portfolio, classs, 'All', 'InvestedEUR'] = 0
            df[portfolio, classs, 'All', 'PnLEUR'] = 0
        else:
            df[portfolio, classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'InvestedEUR']].sum(axis=1)
            df[portfolio, classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'ValueEUR']].sum(axis=1)
            df[portfolio, classs, 'All', 'PnLEUR'] = df[portfolio, classs, 'All', 'ValueEUR'] - df[portfolio, classs, 'All', 'InvestedEUR']

    df['All', classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'InvestedEUR']].sum(axis=1)
    df['All', classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'ValueEUR']].sum(axis=1)
    df['All', classs, 'All', 'PnLEUR'] = df['All', classs, 'All', 'ValueEUR'] - df['All', classs, 'All', 'InvestedEUR']
print(f"\t... OK")



	Concate class data ...
	... OK


In [46]:
s = df.iloc[-1]
ptf = 'ZEN'
print(s.index)
rows = {}
for asset in np.unique(s.index):
    rows[ptf] = {
        'Value': s.loc[pd.IndexSlice[ptf, ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'ValueEUR']].sum(),
        'Invested': s.loc[pd.IndexSlice[ptf, ['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'InvestedEUR']].sum(),
        'Cash': s.loc[pd.IndexSlice[ptf, 'Cash', 'All', 'ValueEUR']],
        'PnL': s.loc[pd.IndexSlice[ptf, ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'PnLEUR']].sum(),
        'Deposited': -s.loc[pd.IndexSlice[ptf, 'Deposit', 'All', 'InvestedEUR']],
    }
pd.DataFrame(rows).transpose().style.format("{:.0f} €")

MultiIndex([('DMA',         'Cash',   '€',     'Quantity'),
            ('DMA',         'Cash',   '€',    'Operation'),
            ('DMA',         'Cash',   '€',       'Market'),
            ('DMA',         'Cash',   '€',        'Forex'),
            ('DMA',      'Deposit', 'EUR',     'Quantity'),
            ('DMA',      'Deposit', 'EUR',    'Operation'),
            ('DMA',      'Deposit', 'EUR',       'Market'),
            ('DMA',      'Deposit', 'EUR', 'OperationEUR'),
            ('DMA',      'Deposit', 'EUR',     'Position'),
            ('DMA',      'Deposit', 'EUR',     'Invested'),
            ...
            ('All',     'Equities', 'All',       'PnLEUR'),
            ('DMA', 'Fixed Income', 'All',     'ValueEUR'),
            ('DMA', 'Fixed Income', 'All',  'InvestedEUR'),
            ('DMA', 'Fixed Income', 'All',       'PnLEUR'),
            ('ZEN', 'Fixed Income', 'All',  'InvestedEUR'),
            ('ZEN', 'Fixed Income', 'All',     'ValueEUR'),
            ('ZEN', 'Fix

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [17]:
def format_columns(df, assets):
    for column in df.columns:
        asset = column[2]
        metric = column[3]
        if metric in ['Operation', 'OperationEUR', 'Invested', 'InvestedEUR', 'Value', 'ValueEUR', 'PnL', 'PnLEUR']:
            df[column] = df[column].map(assets.loc[asset]['ValueFmt'].format)
        if metric in ['Quantity', 'Position']:
            df[column] = df[column].map(assets.loc[asset]['PositionFmt'].format)
        if metric in ['Market', 'PRU']:
            df[column] = df[column].map(assets.loc[asset]['PriceFmt'].format)
    return df


In [18]:
dates = ["2021-04-30","2021-07-28","2021-07-29","2021-07-30","2021-08-10","2021-10-01","2021-11-01","2021-12-01","2021-12-31","2022-02-01","2022-03-24","2022-04-01","2022-04-05","2022-04-13","2022-05-02","2022-04-19","2022-06-01","2022-07-01","2022-07-04","2022-07-05","2022-08-05","2022-08-25","2022-10-12","2023-02-10"]

df['ZEN', 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice['ZEN', :, 'All', 'InvestedEUR']].sum(axis=1)
df.loc[dates, pd.IndexSlice['ZEN', ['Equities'], :, ['Operation', 'Forex', 'OperationEUR', 'Invested', 'InvestedEUR']]]

ZEN                                                      \
            Equities                                                       
                 ESE GREENBULL   ESE GREENBULL          ESE    GREENBULL   
           Operation Operation Forex     Forex OperationEUR OperationEUR   
2021-04-30       0.0     598.0   1.0  1.212709          0.0   493.110802   
2021-07-28       0.0       0.0   1.0  1.182313          0.0     0.000000   
2021-07-29       0.0       0.0   1.0  1.184596          0.0     0.000000   
2021-07-30       0.0     585.0   1.0  1.189300          0.0   491.886106   
2021-08-10       0.0       0.0   1.0  1.173847          0.0     0.000000   
2021-10-01       0.0     575.0   1.0  1.157448          0.0   496.782759   
2021-11-01       0.0     575.0   1.0  1.155668          0.0   497.547525   
2021-12-01       0.0     600.0   1.0  1.133029          0.0   529.553973   
2021-12-31       0.0     500.0   1.0  1.132503          0.0   441.500014   
2022-02-01       0.0     555.0   1.0  1.122965          0.0   494.227503   
2022-03-24       0.0       0.0   1.0  1.100631          0.0     0.000000   
2022-04-01       0.0     530.0   1.0  1.107236          0.0   478.669499   
2022-04-05       0.0       0.0   1.0  1.097586          0.0     0.000000   
2022-04-13       0.0       0.0   1.0  1.082614          0.0     0.000000   
2022-05-02       0.0       0.0   1.0  1.053596          0.0     0.000000   
2022-04-19       0.0       0.0   1.0  1.078179          0.0     0.000000   
2022-06-01       0.0     510.0   1.0  1.073411          0.0   475.121116   
2022-07-01       0.0       0.0   1.0  1.047768          0.0     0.000000   
2022-07-04       0.0       0.0   1.0  1.043395          0.0     0.000000   
2022-07-05       0.0       0.0   1.0  1.043319          0.0     0.000000   
2022-08-05       0.0       0.0   1.0  1.024779          0.0     0.000000   
2022-08-25       0.0       0.0   1.0  0.996910          0.0     0.000000   
2022-10-12     986.4       0.0   1.0  0.970017        986.4     0.000000   
2023-02-10       0.0       0.0   1.0  1.074229          0.0     0.000000   

                                                                     
                                                                     
                ESE GREENBULL         ESE    GREENBULL          All  
           Invested  Invested InvestedEUR  InvestedEUR  InvestedEUR  
2021-04-30      0.0     598.0         0.0   493.110802   493.110802  
2021-07-28      0.0     598.0         0.0   493.110802   493.110802  
2021-07-29      0.0     598.0         0.0   493.110802   493.110802  
2021-07-30      0.0    1183.0         0.0   984.996908   984.996908  
2021-08-10      0.0    1183.0         0.0   984.996908   984.996908  
2021-10-01      0.0    1758.0         0.0  1481.779667  1481.779667  
2021-11-01      0.0    2333.0         0.0  1979.327191  1979.327191  
2021-12-01      0.0    2933.0         0.0  2508.881164  2508.881164  
2021-12-31      0.0    3433.0         0.0  2950.381179  2950.381179  
2022-02-01      0.0    3988.0         0.0  3444.608681  3444.608681  
2022-03-24      0.0    3988.0         0.0  3444.608681  3444.608681  
2022-04-01      0.0    4518.0         0.0  3923.278180  3923.278180  
2022-04-05      0.0    4518.0         0.0  3923.278180  3923.278180  
2022-04-13      0.0    4518.0         0.0  3923.278180  3923.278180  
2022-05-02      0.0    4518.0         0.0  3923.278180  3923.278180  
2022-04-19      0.0    4518.0         0.0  3923.278180  3923.278180  
2022-06-01      0.0    5028.0         0.0  4398.399296  4398.399296  
2022-07-01      0.0    5028.0         0.0  4398.399296  4398.399296  
2022-07-04      0.0    5028.0         0.0  4398.399296  4398.399296  
2022-07-05      0.0    5028.0         0.0  4398.399296  4398.399296  
2022-08-05      0.0    5028.0         0.0  4398.399296  4398.399296  
2022-08-25      0.0    5028.0         0.0  4398.399296  4398.399296  
2022-10-12    986.4    5028.0       986.4  4398.399296  5384.799296 

In [33]:
s = df['ZEN'].iloc[-1]
rows = {}
rows['Value'] = s.loc[pd.IndexSlice[['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'ValueEUR']].sum()
rows['Invested'] = s.loc[pd.IndexSlice[['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'InvestedEUR']].sum()
rows['Cash'] = s.loc[pd.IndexSlice['Cash', 'All', 'ValueEUR']]
rows['Deposited'] = -s.loc[pd.IndexSlice['Deposit', 'All', 'InvestedEUR']]
rows['Deposited2'] = s.loc[pd.IndexSlice[['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'InvestedEUR']].sum()
rows['Deposited3'] = rows['Invested'] + rows['Cash'] - s.loc['Cash', 'All', 'PnLEUR']
rows['PnL'] = s.loc[pd.IndexSlice[['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'PnLEUR']].sum()
rows['PnL2'] = rows['Value'] - rows['Invested'] - (rows['Cash'] - s.loc['Cash', 'All', 'PnLEUR'])
rows['PnL3'] = rows['Value'] - rows['Deposited']

pd.Series(rows).map('{:.0f}'.format)

Value         11643
Invested       9673
Cash           2390
Deposited     12000
Deposited2    12000
Deposited3    12000
PnL            -357
PnL2           -357
PnL3           -357
dtype: object

In [1]:
from api_portfolio import build_data
data, assets = build_data()


    # ----------------------------------------------
    # Get data, concate, merge and extract metrics
    # for portfolio Dashboard 
    # ----------------------------------------------
    
              Market Currency     Forex           Class  PriceFmt PositionFmt  \
Asset                                                                           
EUR         EUREUR=X           EUREUR=X         Deposit                         
€           EUREUR=X      EUR  EUREUR=X            Cash  {:.0f} €      {:.0f}   
$              EUR=X      EUR  EUREUR=X            Cash  {:.4f} €      {:.0f}   
USDT           EUR=X     EUR   EUREUR=X            Cash  {:.4f} €      {:.0f}   
ESE           ESE.PA        €  EUREUR=X        Equities  {:.2f} €      {:.0f}   
PUST                        €  EUREUR=X        Equities  {:.2f} €      {:.0f}   
RS2K                        €  EUREUR=X        Equities  {:.2f} €      {:.0f}   
OBLI         OBLI.PA        €  EUREUR=X    Fixed Income  {:.2f} €      {:.0f}

In [27]:
import pandas as pd
s = data.iloc[-1]
data['2021-07-28':'2021-09-01'].loc[:, pd.IndexSlice['ZEN', 'Cash', ['$'], ['Value', 'Operation', 'Invested', 'PnL']]]

ZEN                               
                 Cash                               
                    $                               
                Value Operation   Invested       PnL
2021-07-28   0.000000  0.000000   6.889211 -6.889211
2021-07-29   0.000000  0.000000   6.889211 -6.889211
2021-07-30  10.930803  8.113876  15.003087 -4.072284
2021-07-31  10.930803  0.000000  15.003087 -4.072284
2021-08-01  10.930803  0.000000  15.003087 -4.072284
2021-08-02  10.954710  0.000000  15.003087 -4.048377
2021-08-03  10.947690  0.000000  15.003087 -4.055397
2021-08-04  10.952500  0.000000  15.003087 -4.050587
2021-08-05  10.982140  0.000000  15.003087 -4.020947
2021-08-06  10.983700  0.000000  15.003087 -4.019387
2021-08-07  10.983700  0.000000  15.003087 -4.019387
2021-08-08  10.983700  0.000000  15.003087 -4.019387
2021-08-09  11.064300  0.000000  15.003087 -3.938787
2021-08-10  11.074700  0.000000  15.003087 -3.928387
2021-08-11  11.089000  0.000000  15.003087 -3.914087
2021-08-12  11.071463  0.000000  15.003087 -3.931624
2021-08-13  11.076780  0.000000  15.003087 -3.926307
2021-08-14  11.076780  0.000000  15.003087 -3.926307
2021-08-15  11.076780  0.000000  15.003087 -3.926307
2021-08-16  11.021920  0.000000  15.003087 -3.981167
2021-08-17  11.035700  0.000000  15.003087 -3.967387
2021-08-18  11.101870  0.000000  15.003087 -3.901217
2021-08-19  11.107980  0.000000  15.003087 -3.895107
2021-08-20  11.133382  0.000000  15.003087 -3.869705
2021-08-21  11.133382  0.000000  15.003087 -3.869705
2021-08-22  11.133382  0.000000  15.003087 -3.869705
2021-08-23  11.113440  0.000000  15.003087 -3.889647
2021-08-24  11.068980  0.000000  15.003087 -3.934107
2021-08-25  11.060920  0.000000  15.003087 -3.942167
2021-08-26  11.040900  0.000000  15.003087 -3.962187
2021-08-27  11.057800  0.000000  15.003087 -3.945287
2021-08-28  11.057800  0.000000  15.003087 -3.945287
2021-08-29  11.057800  0.000000  15.003087 -3.945287
2021-08-30  11.017877  0.000000  15.003087 -3.985210
2021-08-31  11.019840  0.000000  15.003087 -3.983247
2021-09-01  11.007360  0.000000  15.003087 -3.995727

In [69]:
data.iloc[-1].loc[pd.IndexSlice['All', ['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'InvestedEUR']].index
data['All',    'Commodities', 'All', 'InvestedEUR'].values

array([   0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0

In [36]:
s = data.iloc[-1]
rows = {}
for (ptf, classs, asset, _) in s.loc[pd.IndexSlice[ptf, ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], :, 'Market']].index:
    print(ptf, classs, asset, assets.loc[asset]['PriceFmt'], assets.loc[asset]['PriceFmt'].format(s[ptf, classs, asset, 'Market']))
    rows[classs, asset] = {
        'Market': assets.loc[asset]['PriceFmt'].format(s[ptf, classs, asset, 'Market']),
        'PRU': assets.loc[asset]['PriceFmt'].format(s[ptf, classs, asset, 'PRU']),
        'Position': assets.loc[asset]['PositionFmt'].format(s[ptf, classs, asset, 'Position']),
        'Value': assets.loc[asset]['ValueFmt'].format(s[ptf, classs, asset, 'Value']),
        'Invested': assets.loc[asset]['ValueFmt'].format(s[ptf, classs, asset, 'Invested']),
        'PnL': assets.loc[asset]['ValueFmt'].format(s[ptf, classs, asset, 'PnL']),
    }
pd.DataFrame(rows).transpose()

ZEN Cash $ {:.4f} € 0.9414 €
ZEN Cash USDT {:.4f} € 0.9414 €
ZEN Cash € {:.0f} € 1 €
ZEN Commodities GOLD {:.2f} € 68.68 €
ZEN Cryptocurrency BTC {:.0f} $ 24470 $
ZEN Equities ESE {:.2f} € 18.42 €
ZEN Equities GREENBULL {:.0f} $ 12915 $
ZEN Fixed Income OBLI {:.2f} € 8.89 €


,Market,PRU,Position,Value,Invested,PnL
$,0.9414 €,1.2543 €,81,76 €,102 €,-25 €
USDT,0.9414 €,0.8048 €,660,621 €,531 €,90 €
€,1 €,1 €,1694,1694 €,1694 €,0 €
GOLD,68.68 €,69.81 €,26,1786 €,1815 €,-29 €
BTC,24470 $,27875 $,0.0552,1352 $,1540 $,-188 $
ESE,18.42 €,17.93 €,55,1013.12 €,986.40 €,26.72 €
GREENBULL,12915 $,14523 $,0.346,4471 $,5028 $,-557 $
OBLI,8.89 €,9.05 €,111,987 €,1004 €,-17 €
